# Street2Sat Point Analysis

<a target="_blank" href="https://colab.research.google.com/github/nasaharvest/street2sat/blob/scaling-distance-estimation/notebooks/deployment_3_analysis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Author:** Ivan Zvonkov

**Last Edited:** November 20, 2023

**Description:** Notebook for analyzing field points generated using street2sat pipeline. Associated Google Sheet: https://docs.google.com/spreadsheets/d/1SErjKizF6fM_48p8onQwJXWFr07rbqL2ToKh87TvDe8/edit?usp=sharing

**Requirement**: street2sat-v2-points-in-field.csv uploaded into `/content`

In [ ]:
# Set MAPBOX token, for visualizing points on map
MAPBOX_TOKEN = ""
assert MAPBOX_TOKEN != "", "Set MAPBOX_TOKEN above"

In [ ]:
# Login to Google Cloud, for downloading images and predictions
!gcloud auth login
!gcloud config set project "bsos-geog-harvest1"

In [ ]:
# Setup
import folium
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [ ]:
df_path = Path("/content/street2sat-v2-points-in-field.csv")
assert df_path.exists(), f"Upload file {df_path.name} to Google Colab to proceed."
df = pd.read_csv(df_path)

In [ ]:
keys = [
    "input_img",
    "mapbox_img_source",
    "bing_img_source",
    "mapbox_img_segmentation",
    "bing_img_segmentation"
]

def download_images(row: dict):
    img_root = Path(f"/content/{row['name']}")
    img_root.mkdir(exist_ok=True)
    for k in keys:
        path = img_root / f"{k}.jpg"
        if not path.exists():
            img_src = row[k]
            !gcloud storage cp {img_src} {path}
    return img_root

def plot_images(img_root: Path):
    plt.imshow(plt.imread(img_root / (keys[0] + ".jpg")))
    plt.yticks([])
    plt.xticks([])

    fig, ax = plt.subplots(2,2, figsize=(6,6))

    ax[0, 0].imshow(plt.imread(img_root / (keys[1] + ".jpg")))
    ax[1, 0].imshow(plt.imread(img_root / (keys[2] + ".jpg")))
    ax[0, 1].imshow(plt.imread(img_root / (keys[3] + ".jpg")))
    ax[1, 1].imshow(plt.imread(img_root / (keys[4] + ".jpg")))

    for i, row_title in enumerate(["Mapbox", "Bing"]):
        ax[i, 0].set_ylabel(row_title, fontsize=12)

    for j, col_title in enumerate(["Satellite Image", "Segmentation"]):
        ax[1, j].set_xlabel(col_title, fontsize=12)

    for row in ax:
        for col in row:
            col.set_yticks([])
            col.set_yticklabels([])
            col.set_xticks([])
            col.set_xticklabels([])

    plt.tight_layout()


def generate_map(row):
    m = folium.Map(
        location=eval(row["coord"]),
        zoom_start=18,
        tiles='https://api.tiles.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.png?access_token=' + MAPBOX_TOKEN,
        attr='mapbox.com'
    )

    folium.Marker(eval(row["coord"]), popup='GoPro', icon=folium.Icon(color="lightgray")).add_to(m)
    folium.Marker(eval(row["mapbox_field_coord"]), popup='Mapbox', icon=folium.Icon(color="blue")).add_to(m)
    folium.Marker(eval(row["bing_field_coord"]), popup='Bing', icon=folium.Icon(color="orange")).add_to(m)
    return m

def download_and_plot(row):
    img_root = download_images(row)
    print(f"Name: {row['name']}")
    print(f"Prediction: {row['dominant_crop']}")
    plot_images(img_root)


In [ ]:
# Select field points in dataframe to visualize
row = df.iloc[-10]

In [ ]:
download_and_plot(row)

In [ ]:
generate_map(row)